# Matris Faktörizasyon Algoritmaları - Detaylı Analiz

Bu notebook, matris faktörizasyon algoritmalarının detaylı analizini ve görselleştirmelerini içerir.

## İçindekiler
1. SVD - Öneri Sistemi
2. SVD - Gürültü Temizleme
3. PCA - Veri Görselleştirme
4. NMF - Görüntü İşleme
5. NMF - Topic Modeling
6. ALS - Öneri Sistemi
7. Performans Karşılaştırması


In [ ]:
# Gerekli kütüphaneleri import et
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML

# Algoritmalar
from algorithms.svd import SVDRecommender, SVDNoiseReducer
from algorithms.pca import PCAAnalyzer
from algorithms.nmf import NMFImageProcessor, NMFTopicModeler
from algorithms.als import ALSRecommender

# Yardımcı fonksiyonlar
from utils.data_loader import (
    generate_sample_data, 
    generate_rating_matrix,
    load_sample_images,
    generate_text_corpus,
    generate_noisy_data
)
from utils.visualization import (
    plot_ratings_matrix,
    plot_recommendations,
    plot_image_grid,
    plot_topic_words
)

# Görselleştirme ayarları
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

print("✅ Tüm kütüphaneler başarıyla yüklendi!")


## 1. SVD - Öneri Sistemi

### 📚 SVD (Singular Value Decomposition) Nedir?

**SVD**, bir matrisi üç matrisin çarpımına ayıran matematiksel bir yöntemdir:
- **U**: Sol tekil vektörler matrisi (kullanıcı özellikleri)
- **Σ**: Tekil değerler matrisi (köşegen matris, bileşen önemleri)
- **V^T**: Sağ tekil vektörler matrisi (ürün özellikleri)

### 🎯 Öneri Sistemlerinde Kullanımı

1. **Rating Matrisi Faktörizasyonu**: Kullanıcı-ürün rating matrisini düşük rank matrislere ayırır
2. **Latent Faktörler**: Gizli kullanıcı ve ürün özelliklerini keşfeder
3. **Eksik Rating Tahmini**: Kullanıcıların henüz değerlendirmediği ürünler için rating tahmin eder

### 📊 Metrikler

- **RMSE (Root Mean Square Error)**: Tahmin hatasının ölçüsü. Düşük değer = daha iyi performans
- **Tekil Değerler**: Her bileşenin önemini gösterir. Büyük değerler = daha önemli bileşenler


In [ ]:
# Örnek rating matrisi oluştur
print("Rating matrisi oluşturuluyor...")
rating_matrix = generate_rating_matrix(n_users=100, n_items=50, sparsity=0.7)
print(f"Rating matrisi boyutu: {rating_matrix.shape}")
print(f"Dolu rating sayısı: {np.sum(~np.isnan(rating_matrix))}")

# Train-test split
np.random.seed(42)
mask = ~np.isnan(rating_matrix)
test_indices = np.random.choice(np.where(mask)[0], 
                               size=int(0.2 * np.sum(mask)), 
                               replace=False)
test_mask = np.zeros_like(mask, dtype=bool)
test_mask.flat[test_indices] = True

train_matrix = rating_matrix.copy()
train_matrix[test_mask] = np.nan

test_matrix = np.full_like(rating_matrix, np.nan)
test_matrix[test_mask] = rating_matrix[test_mask]

# SVD modeli eğit
print("\nSVD modeli eğitiliyor...")
svd_model = SVDRecommender(n_components=20)
svd_model.fit(train_matrix)

# Değerlendirme
rmse = svd_model.evaluate(test_matrix)
singular_values = svd_model.get_singular_values()

print(f"\nTest RMSE: {rmse:.4f}")
print(f"Kullanılan tekil değer sayısı: {len(singular_values)}")
print(f"İlk 5 tekil değer: {singular_values[:5]}")


### 📈 Tekil Değerler Görselleştirmesi

**Bu grafik ne gösteriyor?**
- **X Ekseni**: Bileşen numarası (1, 2, 3, ...)
- **Y Ekseni**: Tekil değer (bileşenin önemi)
- **Yüksek değerler**: Daha önemli, daha fazla bilgi taşıyan bileşenler
- **Düşük değerler**: Daha az önemli, gürültü içeren bileşenler
- **Elbow Point**: Optimal bileşen sayısını belirler (eğrinin düşmeye başladığı nokta)

**Kümülatif Varyans Grafiği**:
- İlk N bileşenin toplam açıkladığı varyans
- %95 eşiği: Verinin %95'ini korumak için gerekli bileşen sayısı


In [ ]:
# Tekil değerler görselleştirme
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Tekil değerler
ax1.plot(range(1, len(singular_values) + 1), singular_values, 'o-', linewidth=2, markersize=6)
ax1.set_xlabel('Bileşen', fontsize=12)
ax1.set_ylabel('Tekil Değer', fontsize=12)
ax1.set_title('Tekil Değerler (Singular Values)', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Kümülatif varyans
cumulative_variance = np.cumsum(singular_values**2) / np.sum(singular_values**2)
ax2.plot(range(1, len(singular_values) + 1), cumulative_variance, 'o-', linewidth=2, markersize=6)
ax2.axhline(y=0.95, color='r', linestyle='--', label='95% Eşiği')
ax2.set_xlabel('Bileşen Sayısı', fontsize=12)
ax2.set_ylabel('Kümülatif Varyans Oranı', fontsize=12)
ax2.set_title('Kümülatif Varyans', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


### 🎯 Öneriler ve Rating Matrisi

**Öneri Sistemi Çalışma Prensibi**:
1. Model, kullanıcının geçmiş rating'lerini analiz eder
2. Latent faktörler kullanarak kullanıcı tercihlerini öğrenir
3. Benzer kullanıcıların beğendiği ürünleri bulur
4. Kullanıcının henüz görmediği ürünler için rating tahmin eder
5. En yüksek tahmin edilen rating'lere sahip ürünleri önerir

**Rating Matrisi Görselleştirmesi**:
- **Satırlar**: Kullanıcılar
- **Sütunlar**: Ürünler
- **Renkler**: Rating değerleri (koyu = düşük, açık = yüksek)
- **Beyaz alanlar**: Eksik rating'ler (kullanıcı henüz değerlendirmemiş)


In [ ]:
# Öneriler üret
user_idx = 0
predictions = svd_model.predict_all()[user_idx]
rated_items = ~np.isnan(rating_matrix[user_idx])
predictions[rated_items] = -np.inf

top_items = np.argsort(predictions)[::-1][:10]
top_ratings = predictions[top_items]

# Öneri görselleştirme
fig = plot_recommendations(user_idx, (top_items, top_ratings))
plt.show()

# Rating matrisi görselleştirme
fig2 = plot_ratings_matrix(rating_matrix)
plt.show()


## 2. SVD - Gürültü Temizleme

### 🔇 SVD ile Gürültü Temizleme Nasıl Çalışır?

**Prensip**: SVD, veriyi önemli bileşenlere ve gürültüye ayırır.

**Adımlar**:
1. **SVD Uygulama**: Veri matrisini tekil değerlerine ayırır
2. **Bileşen Seçimi**: Sadece önemli bileşenleri tutar (yüksek tekil değerler)
3. **Gürültü Filtreleme**: Düşük tekil değerli bileşenleri atar (gürültü)
4. **Yeniden Oluşturma**: Seçilen bileşenlerle temiz veriyi yeniden oluşturur

### 📊 Varyans Eşiği

- **0.95 (95%)**: Verinin %95'ini korur, %5 gürültüyü temizler
- **0.90 (90%)**: Daha agresif temizleme, daha fazla gürültü kaldırır
- **0.99 (99%)**: Çok az temizleme, neredeyse tüm veriyi korur


In [ ]:
# Örnek veri oluştur
X, _ = generate_sample_data(n_samples=200, n_features=100, n_clusters=5)
X_noisy = generate_noisy_data(X, noise_level=0.2)

# SVD ile gürültü temizleme
noise_reducer = SVDNoiseReducer(n_components=None, threshold=0.95)
noise_reducer.fit(X_noisy)
X_denoised = noise_reducer.denoise(X_noisy)

# Metrikler
mse_original = np.mean((X - X_noisy)**2)
mse_denoised = np.mean((X - X_denoised)**2)
improvement = ((mse_original - mse_denoised) / mse_original * 100)
noise_reduction_ratio = noise_reducer.get_noise_reduction_ratio()

print(f"Gürültü MSE: {mse_original:.6f}")
print(f"Temizlenmiş MSE: {mse_denoised:.6f}")
print(f"İyileştirme: {improvement:.2f}%")
print(f"Varyans korunma oranı: {noise_reduction_ratio:.2%}")
print(f"Kullanılan bileşen sayısı: {noise_reducer.n_components}")


In [ ]:
# Gürültü temizleme görselleştirme
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Orijinal
im1 = axes[0].imshow(X[:50, :50], cmap='viridis', aspect='auto')
axes[0].set_title('Orijinal Veri', fontsize=14, fontweight='bold')
plt.colorbar(im1, ax=axes[0])

# Gürültülü
im2 = axes[1].imshow(X_noisy[:50, :50], cmap='viridis', aspect='auto')
axes[1].set_title('Gürültülü Veri', fontsize=14, fontweight='bold')
plt.colorbar(im2, ax=axes[1])

# Temizlenmiş
im3 = axes[2].imshow(X_denoised[:50, :50], cmap='viridis', aspect='auto')
axes[2].set_title('Temizlenmiş Veri', fontsize=14, fontweight='bold')
plt.colorbar(im3, ax=axes[2])

plt.tight_layout()
plt.show()

# Varyans analizi
component_counts, variance_ratios = noise_reducer.get_optimal_components(X_noisy)

fig2, ax = plt.subplots(figsize=(10, 5))
ax.plot(component_counts, variance_ratios, 'o-', linewidth=2, markersize=6)
ax.axhline(y=0.95, color='r', linestyle='--', label='95% Eşiği')
ax.axvline(x=noise_reducer.n_components, color='g', 
          linestyle='--', label=f'Seçilen ({noise_reducer.n_components})')
ax.set_xlabel('Bileşen Sayısı', fontsize=12)
ax.set_ylabel('Kümülatif Varyans Oranı', fontsize=12)
ax.set_title('Varyans Korunma Analizi', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


## 3. PCA - Veri Görselleştirme

### 📈 PCA (Principal Component Analysis) Nedir?

**PCA**, yüksek boyutlu veriyi daha düşük boyutlu bir uzaya izdüşüren bir boyut azaltma tekniğidir.

### 🔧 Nasıl Çalışır?

1. **Kovaryans Matrisi**: Veri özellikleri arasındaki ilişkileri hesaplar
2. **Özvektörler**: Verinin ana yönlerini (principal components) bulur
3. **Özdeğerler**: Her bileşenin ne kadar varyans açıkladığını gösterir
4. **İzdüşüm**: Veriyi yeni düşük boyutlu uzaya dönüştürür

### 🎯 Kullanım Alanları

- **Veri Görselleştirme**: Yüksek boyutlu veriyi 2D/3D'de görselleştirme
- **Özellik Seçimi**: En önemli özellikleri belirleme
- **Boyut Azaltma**: Gürültüyü azaltma ve hesaplama hızını artırma
- **Ön İşleme**: Makine öğrenmesi modelleri için veri hazırlama


In [ ]:
# Örnek veri oluştur
X, y = generate_sample_data(n_samples=500, n_features=50, n_clusters=5)

# PCA uygula
pca = PCAAnalyzer(n_components=None)
X_transformed = pca.fit_transform(X)

# Analiz
n_95 = pca.get_optimal_components(X, variance_threshold=0.95)
variance_first_5 = np.sum(pca.explained_variance_ratio_[:5])

print(f"Toplam bileşen sayısı: {len(pca.explained_variance_ratio_)}")
print(f"%95 varyans için gerekli bileşen sayısı: {n_95}")
print(f"İlk 5 bileşenin açıkladığı varyans: {variance_first_5:.2%}")


In [ ]:
# Açıklanan varyans grafiği
fig = pca.plot_explained_variance(n_components=20)
plt.show()

# 2D ve 3D izdüşümler
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))

# 2D
X_2d = X_transformed[:, :2]
scatter1 = ax1.scatter(X_2d[:, 0], X_2d[:, 1], c=y, cmap='viridis', alpha=0.6, s=50)
ax1.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} varyans)', fontsize=12)
ax1.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} varyans)', fontsize=12)
ax1.set_title('PCA 2D İzdüşümü', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=ax1)

# 3D
from mpl_toolkits.mplot3d import Axes3D
ax2 = fig.add_subplot(122, projection='3d')
X_3d = X_transformed[:, :3]
scatter2 = ax2.scatter(X_3d[:, 0], X_3d[:, 1], X_3d[:, 2], 
                      c=y, cmap='viridis', alpha=0.6, s=50)
ax2.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%})', fontsize=10)
ax2.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%})', fontsize=10)
ax2.set_zlabel(f'PC3 ({pca.explained_variance_ratio_[2]:.2%})', fontsize=10)
ax2.set_title('PCA 3D İzdüşümü', fontsize=14, fontweight='bold')
plt.colorbar(scatter2, ax=ax2)

plt.tight_layout()
plt.show()


## 4. NMF - Görüntü İşleme

### 🖼️ NMF (Non-negative Matrix Factorization) Nedir?

**NMF**, matrisleri sadece **pozitif değerlerle** faktörize eden bir yöntemdir.

### 🎨 Görüntü İşlemede Kullanımı

1. **Basis Images (Temel Görüntüler)**: Görüntülerin temel yapı taşlarını bulur
2. **Katsayı Matrisi**: Her görüntünün bu temel yapı taşlarını nasıl kullandığını gösterir
3. **Yeniden Oluşturma**: Temel görüntüler ve katsayılarla orijinal görüntüyü yeniden oluşturur

### ✅ Avantajları

- **Pozitif Değerler**: Görüntü piksel değerleri doğal olarak pozitiftir
- **Yorumlanabilirlik**: Temel görüntüler anlamlı pattern'ler içerir
- **Sıkıştırma**: Az sayıda temel görüntü ile çok görüntüyü temsil edebilir
- **Gürültü Azaltma**: Düşük rank yaklaşımı gürültüyü filtreler


In [ ]:
# Görüntü verisi yükle
images_flat, image_shape = load_sample_images(n_images=100)

# NMF modeli
nmf_image = NMFImageProcessor(n_components=20)
nmf_image.fit(images_flat)

# Yeniden oluştur
reconstructed = nmf_image.reconstruct()
basis_images = nmf_image.get_basis_images(image_shape)
compression_ratio = nmf_image.get_compression_ratio(image_shape)

# Metrikler
mse = np.mean((images_flat - reconstructed)**2)

print(f"Sıkıştırma oranı: {compression_ratio:.2f}x")
print(f"MSE: {mse:.6f}")
print(f"Basis görüntü sayısı: {len(basis_images)}")


In [ ]:
# Görüntü görselleştirme
fig, axes = plt.subplots(3, 5, figsize=(15, 9))

for i in range(5):
    # Orijinal
    axes[0, i].imshow(images_flat[i].reshape(image_shape), cmap='gray')
    axes[0, i].set_title(f'Orijinal {i+1}')
    axes[0, i].axis('off')
    
    # Yeniden oluşturulmuş
    axes[1, i].imshow(reconstructed[i].reshape(image_shape), cmap='gray')
    axes[1, i].set_title(f'Yeniden Oluşturulmuş {i+1}')
    axes[1, i].axis('off')
    
    # Fark
    diff = np.abs(images_flat[i].reshape(image_shape) - 
                 reconstructed[i].reshape(image_shape))
    axes[2, i].imshow(diff, cmap='hot')
    axes[2, i].set_title(f'Fark {i+1}')
    axes[2, i].axis('off')

plt.suptitle('Orijinal vs Yeniden Oluşturulmuş Görüntüler', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

# Basis görüntüleri
basis_flat = basis_images.reshape(len(basis_images), -1)
fig2 = plot_image_grid(basis_flat, image_shape, n_cols=5, figsize=(15, 8))
plt.suptitle('Temel Görüntüler (Basis Images)', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()


## 5. NMF - Topic Modeling

### 📝 Topic Modeling Nedir?

**Topic Modeling**, metin dokümanlarında gizli konuları (topic) keşfetme tekniğidir.

### 🔧 NMF ile Topic Modeling

1. **TF-IDF Vektörizasyon**: Metinleri sayısal vektörlere dönüştürür
2. **NMF Faktörizasyon**: Doküman-kelime matrisini faktörize eder
3. **Topic Çıkarımı**: Her topic için önemli kelimeleri bulur
4. **Doküman-Topic Dağılımı**: Her dokümanın hangi topic'lere ait olduğunu gösterir

### 🎯 Kullanım Alanları

- Haber kategorilendirme
- Müşteri yorumları analizi
- Araştırma makaleleri sınıflandırma
- Sosyal medya içerik analizi


In [ ]:
# Metin korpusu oluştur
documents = generate_text_corpus(n_documents=200)

# NMF modeli
nmf_model = NMFTopicModeler(n_topics=5, max_iter=200)
nmf_model.fit(documents, max_features=500, min_df=2, max_df=0.95)

# Topic'ler
topics = nmf_model.get_topics(n_words=10)
coherence = nmf_model.get_topic_coherence()

print(f"Topic tutarlılığı: {coherence:.4f}\n")

# Topic'leri göster
for topic_name, words_scores in topics.items():
    print(f"{topic_name}:")
    for word, score in words_scores[:5]:
        print(f"  - {word}: {score:.4f}")
    print()


In [ ]:
# Topic görselleştirme
fig = plot_topic_words(topics, n_words=10, figsize=(18, 6))
plt.show()

# Doküman-topic dağılımı
doc_topics = nmf_model.get_document_topics()

fig2, ax = plt.subplots(figsize=(12, 8))
sns.heatmap(doc_topics[:50].T, cmap='YlOrRd', ax=ax, 
           yticklabels=[f'Topic {i+1}' for i in range(nmf_model.n_topics)],
           xticklabels=[f'Doc {i+1}' for i in range(50)])
ax.set_xlabel('Dokümanlar', fontsize=12)
ax.set_ylabel('Topic\'ler', fontsize=12)
ax.set_title('Doküman-Topic Dağılımı (İlk 50 Doküman)', 
            fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


## 6. ALS - Öneri Sistemi

### ⚡ ALS (Alternating Least Squares) Nedir?

**ALS**, büyük ölçekli öneri sistemleri için optimize edilmiş bir matris faktörizasyon yöntemidir.

### 🔧 Nasıl Çalışır?

1. **Alternatif Optimizasyon**: Kullanıcı ve ürün faktörlerini sırayla optimize eder
2. **Paralel İşleme**: Her kullanıcı/ürün bağımsız işlenebilir (Spark uyumlu)
3. **Regularizasyon**: Aşırı öğrenmeyi (overfitting) önler
4. **Iteratif Güncelleme**: Her iterasyonda faktörleri iyileştirir

### 🆚 SVD'den Farkları

- **Paralel Çalışma**: Büyük veri setlerinde daha hızlı
- **Eksik Veri**: Sparse matrislerde daha iyi performans
- **Ölçeklenebilirlik**: Milyonlarca kullanıcı/ürün ile çalışabilir
- **Regularizasyon**: Daha iyi genelleme


In [ ]:
# Örnek rating matrisi
rating_matrix = generate_rating_matrix(n_users=100, n_items=50, sparsity=0.7)

# Train-test split
np.random.seed(42)
mask = ~np.isnan(rating_matrix)
test_indices = np.random.choice(np.where(mask)[0], 
                               size=int(0.2 * np.sum(mask)), 
                               replace=False)
test_mask = np.zeros_like(mask, dtype=bool)
test_mask.flat[test_indices] = True

train_matrix = rating_matrix.copy()
train_matrix[test_mask] = np.nan

test_matrix = np.full_like(rating_matrix, np.nan)
test_matrix[test_mask] = rating_matrix[test_mask]

# ALS modeli
print("ALS modeli eğitiliyor...")
als_model = ALSRecommender(n_factors=20, regularization=0.1, iterations=15)
als_model.fit(train_matrix, implicit=False)

# Değerlendirme
rmse = als_model.evaluate(test_matrix)
print(f"Test RMSE: {rmse:.4f}")


In [ ]:
# Öneriler
user_idx = 0
recommendations = als_model.recommend(user_idx, n_recommendations=10, 
                                     exclude_rated=True, 
                                     rating_matrix=rating_matrix)

print(f"Kullanıcı {user_idx+1} için öneriler:")
for i, (item, rating) in enumerate(zip(recommendations[0][:5], recommendations[1][:5]), 1):
    print(f"  {i}. Ürün {item+1}: {rating:.2f}")

# Benzer item'lar
item_idx = 0
similar_items = als_model.get_similar_items(item_idx, n_similar=10)

print(f"\nÜrün {item_idx+1} ile benzer ürünler:")
for i, (item, similarity) in enumerate(zip(similar_items[0][:5], similar_items[1][:5]), 1):
    print(f"  {i}. Ürün {item+1}: Benzerlik = {similarity:.4f}")


## 7. Performans Karşılaştırması

### 📊 Karşılaştırılan Algoritmalar

**SVD (Singular Value Decomposition)**:
- ✅ Matematiksel olarak en kesin
- ✅ Hızlı eğitim
- ❌ Büyük veri setlerinde yavaş
- ❌ Paralel çalışmaya uygun değil

**ALS (Alternating Least Squares)**:
- ✅ Büyük ölçekli veri setlerinde hızlı
- ✅ Paralel çalışmaya uygun (Spark)
- ✅ Sparse matrislerde iyi
- ❌ Daha fazla iterasyon gerekir

### 📈 Metrikler

**RMSE (Root Mean Square Error)**:
- Tahmin hatasının ölçüsü
- **Düşük RMSE = Daha iyi performans**
- Formül: √(Σ(tahmin - gerçek)² / n)

**Eğitim Süresi**:
- Modelin eğitilmesi için geçen süre
- Büyük veri setlerinde önemli

**Not**: Bu sonuçlar veri setine ve parametrelere bağlıdır. Farklı veri setlerinde sonuçlar değişebilir.


In [ ]:
# SVD ve ALS karşılaştırması
rating_matrix = generate_rating_matrix(n_users=100, n_items=50, sparsity=0.7)

# Train-test split
np.random.seed(42)
mask = ~np.isnan(rating_matrix)
test_indices = np.random.choice(np.where(mask)[0], 
                               size=int(0.2 * np.sum(mask)), 
                               replace=False)
test_mask = np.zeros_like(mask, dtype=bool)
test_mask.flat[test_indices] = True

train_matrix = rating_matrix.copy()
train_matrix[test_mask] = np.nan

test_matrix = np.full_like(rating_matrix, np.nan)
test_matrix[test_mask] = rating_matrix[test_mask]

# SVD
import time
start_time = time.time()
svd_model = SVDRecommender(n_components=20)
svd_model.fit(train_matrix)
svd_train_time = time.time() - start_time
svd_rmse = svd_model.evaluate(test_matrix)

# ALS
start_time = time.time()
als_model = ALSRecommender(n_factors=20, regularization=0.1, iterations=15)
als_model.fit(train_matrix)
als_train_time = time.time() - start_time
als_rmse = als_model.evaluate(test_matrix)

# Sonuçlar
results_df = pd.DataFrame({
    'Algoritma': ['SVD', 'ALS'],
    'RMSE': [svd_rmse, als_rmse],
    'Eğitim Süresi (s)': [svd_train_time, als_train_time]
})

print("Performans Karşılaştırması:")
display(results_df)

# Görselleştirme
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.bar(results_df['Algoritma'], results_df['RMSE'], alpha=0.7, color=['#1f77b4', '#ff7f0e'])
ax1.set_ylabel('RMSE', fontsize=12)
ax1.set_title('RMSE Karşılaştırması', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

ax2.bar(results_df['Algoritma'], results_df['Eğitim Süresi (s)'], 
       alpha=0.7, color=['#1f77b4', '#ff7f0e'])
ax2.set_ylabel('Eğitim Süresi (saniye)', fontsize=12)
ax2.set_title('Eğitim Süresi Karşılaştırması', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()
